# 1. 安装和加载使用Step Functions Data Science SDK必需的模块

In [119]:
import sys
!{sys.executable} -m pip install --upgrade stepfunctions

In [82]:
conda install joblib

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::bokeh==2.2.3=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/noarch::aiobotocore==1.2.1=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-spyder==0.3.2=pyhd8ed1ab_0
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.3.4=py36hd391965_0
  - conda-forge/linux-64::spyder==4.2.0=py36h5fab9bb_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::sphinx==3.5.1=pyhd8ed1ab_0
  - conda-

In [120]:
import sagemaker
import time
import random
import uuid
import logging
import stepfunctions
import io

from stepfunctions import steps
from stepfunctions.steps import TransformStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow
from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath

session = sagemaker.Session()
stepfunctions.set_stream_logger(level=logging.INFO)

##定义Sagemaker的路径
bucket = 'sagemaker-cn-north-1-685095924131'
prefix = 'jsfund'
bucket_path = 's3://sagemaker-cn-north-1-685095924131'

# 2. 创建一个Role附加到Step Functions

## 2.1 关联的策略如下所示，并将策略附加到创建的Role

In [20]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:CreateTransformJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:StopTransformJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DeleteEndpoint",
                "sagemaker:UpdateEndpoint",
                "sagemaker:ListTags",
                "sqs:SendMessage",
                "sns:Publish",
                "ecs:RunTask",
                "ecs:StopTask",
                "ecs:DescribeTasks",
                "glue:StartJobRun",
                "glue:GetJobRun",
                "glue:GetJobRuns",
                "glue:BatchStopJobRun"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['sagemaker:CreateTransformJob',
    'sagemaker:DescribeTransformJob',
    'sagemaker:StopTransformJob',
    'sagemaker:CreateTrainingJob',
    'sagemaker:DescribeTrainingJob',
    'sagemaker:StopTrainingJob',
    'sagemaker:CreateHyperParameterTuningJob',
    'sagemaker:DescribeHyperParameterTuningJob',
    'sagemaker:StopHyperParameterTuningJob',
    'sagemaker:CreateModel',
    'sagemaker:CreateEndpointConfig',
    'sagemaker:CreateEndpoint',
    'sagemaker:DeleteEndpointConfig',
    'sagemaker:DeleteEndpoint',
    'sagemaker:UpdateEndpoint',
    'sagemaker:ListTags',
    'sqs:SendMessage',
    'sns:Publish',
    'ecs:RunTask',
    'ecs:StopTask',
    'ecs:DescribeTasks',
    'glue:StartJobRun',
    'glue:GetJobRun',
    'glue:GetJobRuns',
    'glue:BatchStopJobRun'],
   'Resource': '*'},
  {'Effect': 'Allow',
   'Action': ['iam:PassRole'],
   'Resource': '*',
   'Condition': {'StringEquals': {'iam:PassedToSer

In [95]:
import sagemaker

## SageMaker执行角色
sagemaker_execution_role = sagemaker.get_execution_role()

## 从上方输出结果粘贴 StepFunctionsWorkflowExecutionRole ARN
workflow_execution_role = "arn:aws-cn:iam::685095924131:role/StepFunctionsWorkflowExecutionRole" 

# 3. 读取样本数据和加载模型

In [75]:
!ls

docker
Dockerfile
loss_warning_model.pkl
lost+found
model.tar.gz
sagemaker-batch-transform-processing-demo-user_profile_loss_warning_predict.ipynb
sagemaker-batch-transform-user_profile_loss_warning_predict-Copy1.ipynb
sagemaker-batch-transform-user_profile_loss_warning_predict.ipynb
sample_data.csv
sample_data_noheader.csv
sourcedir.tar.gz
Step-Functions-Data-Science-SDK_Glue_Sagemaker-batch-transform.ipynb
transform_script.py
Untitled.ipynb
使用Step Functions Data Science SDK构建从Glue到Sagemaker流水线-20210504.ipynb


In [76]:
%cd /home/ec2-user/SageMaker/

/home/ec2-user/SageMaker


In [77]:
!pwd 

/home/ec2-user/SageMaker


In [67]:
!pip install xgboost

In [95]:
!pip install Scikit-learn==0.20.0

     |████████████████████████████████| 5.3 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: Scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [96]:
import sklearn
print(sklearn.__version__)

0.20.0


In [97]:
import pandas as pd
import numpy as np
import s3fs

from xgboost import XGBClassifier

from sklearn.externals import joblib
##import joblib

import warnings
warnings.filterwarnings('ignore')

In [98]:
data_sample = pd.read_csv('sample_data.csv')

In [99]:
data_sample

,cust_id,trade_amt_last6m,pur_amt_online_last6m,pur_cnt_online_last6m,trade_amt_last3m,redem_cnt_online_last6m,pur_amt_last6m_growth,pur_amt_last6m_main,pur_amt_last1y_bond,mv_ttl_bond_type_avg_last3m,...,zz_cnt,pur_amt_last1m_stock,ret_rate_last2y_bond,pur_amt_last1y_award,fund_cnt_stock_type,pur_cnt_last1y_award,redem_amt_last1y_growth,redem_cnt_last1y_growth,pur_cnt_last6m_alltrack,p_dt
0,1069553680,0.00,0.0,0,0.00,0,0.0,0.0,0.0,0.000000,...,0,0.0,0.000000,0.0,1,0,0.00,0,0,20201231
1,1069554929,1500.00,0.0,0,900.00,0,0.0,0.0,0.0,0.000000,...,4,0.0,0.000000,0.0,1,0,11215.68,1,0,20201231
2,1069555867,108483.67,7800.0,3,33341.17,5,5800.0,7800.0,0.0,0.000000,...,5,0.0,0.000000,52800.0,2,22,53884.83,3,0,20201231
3,1069561326,100.00,100.0,1,100.00,0,0.0,0.0,100.0,100.091416,...,0,0.0,0.001918,0.0,0,0,0.00,0,0,20201231
4,1069568341,0.00,0.0,0,0.00,0,0.0,0.0,0.0,47852.314580,...,0,0.0,0.004741,0.0,0,0,0.00,0,0,20201231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1113893142,2800.00,0.0,0,1300.00,0,0.0,0.0,0.0,0.000000,...,2,0.0,0.000000,0.0,1,0,0.00,0,0,20201231
9996,1113894480,910.39,470.0,8,30.00,1,470.0,0.0,0.0,0.000000,...,2,0.0,0.000000,0.0,1,0,440.39,1,0,20201231
9997,1113895116,3750.00,0.0,0,1250.00,0,3750.0,0.0,0.0,0.000000,...,3,400.0,0.000000,0.0,1,0,0.00,0,0,20201231
9998,1113895561,1500.00,1500.0,3,0.00,0,1500.0,1500.0,0.0,0.000000,...,1,0.0,0.000000,1500.0,1,3,0.00,0,3,20201231


In [100]:
xgb_clf = joblib.load('loss_warning_model.pkl')

In [101]:
data_sample_new = data_sample.drop(['cust_id'],axis=1)

In [102]:
data_sample_new = data_sample_new.drop(['p_dt'],axis=1)

In [103]:
y_pred = xgb_clf.predict_proba(data_sample_new.iloc[:,:].values)[:,1]

In [104]:
yy_pred = xgb_clf.predict(data_sample_new.iloc[:,:].values)

# 4. 准备工作 BYOC（执行一次）

## 4.1 打包容器&推送容器到ECR

In [25]:
import os

import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
bucket = 'sagemaker-cn-north-1-685095924131'
prefix = 'jsfund'
role = get_execution_role()

In [ ]:
import os

import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
bucket = 'sagemaker-cn-north-1-123456789012'
prefix = 'demo'
role = get_execution_role()

In [26]:
%%writefile Dockerfile

From 450853457545.dkr.ecr.cn-north-1.amazonaws.com.cn/sagemaker-scikit-learn:0.20.0-cpu-py3
RUN pip install xgboost

Overwriting Dockerfile


In [27]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'jsfund-sagemaker-batch-transform-demo'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
print(processing_repository_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)
print(ecr)
!aws ecr create-repository --repository-name $ecr_repository

685095924131.dkr.ecr.cn-north-1.amazonaws.com.cn/jsfund-sagemaker-batch-transform-demo:latest
685095924131.dkr.ecr.cn-north-1.amazonaws.com.cn

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'jsfund-sagemaker-batch-transform-demo' already exists in the registry with id '685095924131'


In [28]:
# if it said no basic auth for pull base image, use below cli (cn-north-1)
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin 
450853457545.dkr.ecr.cn-north-1.amazonaws.com.cn

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [32]:
%cd ~/SageMaker/docker

/home/ec2-user/SageMaker/docker


In [33]:
!pwd

/home/ec2-user/SageMaker/docker


In [34]:
!sudo aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  2.048kB
Step 1/2 : From 450853457545.dkr.ecr.cn-north-1.amazonaws.com.cn/sagemaker-scikit-learn:0.20.0-cpu-py3
0.20.0-cpu-py3: Pulling from sagemaker-scikit-learn

a89234b4: Pulling fs layer 
26c6b9c9: Pulling fs layer 
bf18aa40: Pulling fs layer 
c688ebe3: Pulling fs layer 
4d223c24: Pulling fs layer 
5e4a2395: Pulling fs layer 
c7405963: Pulling fs layer 
5a1483b6: Pulling fs layer 
e0d7bfac: Pulling fs layer 
4d223c24: Waiting fs layer 
f6891127: Pulling fs layer 
Digest: sha256:0924c8fff44441d5dab74fc8ff7c5ecbc94af2215a4581e3302f850275bb900bA
Status: Downloaded newer image for 450853457545.dkr.ecr.cn-north-1.amazonaws.com.cn/sagemaker-scikit-learn:0.20.0-cpu-py3
 ---> 6c10beb91746
Step 2/2 : RUN pip insta

In [ ]:
image_uri = processing_repository_uri
image_uri

## 4.2 创建模型

In [78]:
# 打包模型
model = 'loss_warning_model.pkl'
model_uri = 's3://{}/{}/model/model.tar.gz'.format(bucket, prefix)

model_name = 'jsfund-model-1'

model_uri

's3://sagemaker-cn-north-1-685095924131/jsfund/model/model.tar.gz'

In [79]:
# 上传到S3
!tar -czf model.tar.gz $model
!aws s3 cp model.tar.gz $model_uri

upload: ./model.tar.gz to s3://sagemaker-cn-north-1-685095924131/jsfund/model/model.tar.gz


## 4.3 推理脚本（打包到SageMaker模型）

In [80]:
%%writefile transform_script.py

import pandas as pd
import numpy as np
import os
from xgboost import XGBClassifier
from sklearn.externals import joblib

def model_fn(model_dir):
    xgb_clf = joblib.load(os.path.join(model_dir,'loss_warning_model.pkl'))
    return xgb_clf

# def input_fn(request_body, request_content_type):
#     csv_data = request_body.split('\n')
#     list1 = []
#     for i in csv_data:
#         if i != '':
#             list1.append(i.split(','))
#     arr = np.array(list1)
#     return arr

def predict_fn(input_data, model):
    pred_prob = model.predict_proba(input_data)[:,1:] #[:,1]
    return pred_prob

Overwriting transform_script.py


In [ ]:
%%writefile transform_script.py

import pandas as pd
import numpy as np
import os
from xgboost import XGBClassifier
from sklearn.externals import joblib

def model_fn(model_dir):
    xgb_clf = joblib.load(os.path.join(model_dir,'loss_warning_model.pkl'))
    return xgb_clf

def predict_fn(input_data, model):
    pred_prob = model.predict_proba(input_data)[:,1:] #[:,1]
    return pred_prob

### 4.3.1 打包代码并上传到S3

In [57]:
transform_script = 'transform_script.py'
code_uri = 's3://{}/{}/code/sourcedir.tar.gz'.format(bucket, prefix)
code_uri

's3://sagemaker-cn-north-1-685095924131/jsfund/code/sourcedir.tar.gz'

In [58]:
!tar -czf sourcedir.tar.gz $transform_script
!aws s3 cp sourcedir.tar.gz $code_uri

upload: ./sourcedir.tar.gz to s3://sagemaker-cn-north-1-685095924131/jsfund/code/sourcedir.tar.gz


## 4.4 创建SageMaker模型

In [81]:
import boto3

client = boto3.client('sagemaker')

In [82]:
model_response = client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image_uri,
        'ImageConfig': {
            'RepositoryAccessMode': 'Platform'
        },
        'Mode': 'SingleModel',
        'ModelDataUrl': model_uri,
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL':'20',
            'SAGEMAKER_PROGRAM': transform_script,
            'SAGEMAKER_REGION': region,
            'SAGEMAKER_SUBMIT_DIRECTORY':code_uri
        },
    },
    ExecutionRoleArn=role,
    EnableNetworkIsolation=False
)

# 5. 每次批量转换需要执行的代码（可执行多次）

## 5.1 使用SageMaker Batch Transform

### 5.1.1 输出结果不整合输入id

In [105]:
from time import gmtime, strftime

# This file doesn't contain header
data_file = 'sample_data_noheader.csv'

batch_job_name = 'Batch-Transform-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#upload sample data to S3
input_location = 's3://{}/{}/input/{}'.format(bucket, prefix, data_file)
output_location = 's3://{}/{}/output/{}'.format(bucket, prefix, batch_job_name)

In [106]:
from sagemaker.transformer import Transformer

##指定model_name,实例的数量和机型
tfm_transformer = Transformer(model_name='jsfund-model-1', 
                  instance_count=1, 
                  instance_type='ml.m4.xlarge', 
                  strategy=None, 
                  accept="text/csv",
                  assemble_with="Line",
                  output_path='s3://{}/{}/output/{}'.format(bucket, prefix, batch_job_name), 
                  max_concurrent_transforms=None, 
                  max_payload=None, 
                  tags=None, 
                  env=None, 
                  base_transform_job_name=batch_job_name, 
                  sagemaker_session=None)

# 6. 创建Step Functions Step

## 6.1 创建Glue Step，指定glue job name

In [107]:
etl_step = steps.GlueStartJobRunStep(
    'Extract, Transform, Load',
    parameters={"JobName":'glue-demo-job-0223',
               }
)

## 6.2 创建Batch Transform Step，指定job name和 model_name

In [108]:
transform_step = steps.TransformStep(
    'BatchPredictionStep',
    tfm_transformer,
    job_name='Batch-Transform-anytime', 
    model_name='jsfund-model-1',
    data='s3://{}/{}/input/{}'.format(bucket, prefix, data_file),
    content_type="text/csv",
    data_type='S3Prefix',
    split_type="Line",
    input_filter = "$[1:72]",
    join_source='Input', 
    output_filter="$[0,-1]"
)

In [ ]:
transform_step = steps.TransformStep(
    'BatchPredictionStep',
    tfm_transformer,
    job_name='Batch-Transform-anytime', 
    model_name='demo-model-1',
    data='s3://{}/{}/input/{}'.format(bucket, prefix, data_file),
    content_type="text/csv",
    data_type='S3Prefix',
    split_type="Line",
    input_filter = "$[1:72]",
    join_source='Input', 
    output_filter="$[0,-1]"
)

## 6.3 串联上述步骤

In [109]:
from stepfunctions.steps.fields import Field

In [92]:
attrs=getattr(etl_step, "fields")
print(attrs)
attrs[Field.Resource.value] = "arn:aws-cn:states:::glue:startJobRun.sync"
setattr(etl_step, "fields", attrs)
print(attrs)

{'parameters': {'JobName': 'glue-demo-job-0223'}, 'resource': 'arn:aws:states:::glue:startJobRun.sync', 'type': 'Task'}
{'parameters': {'JobName': 'glue-demo-job-0223'}, 'resource': 'arn:aws-cn:states:::glue:startJobRun.sync', 'type': 'Task'}


In [57]:
attrs=getattr(transform_step, "fields")
attrs[Field.Resource.value] = "arn:aws-cn:states:::sagemaker:createTransformJob.sync"
setattr(transform_step, "fields", attrs)
print(attrs)

{'resource': 'arn:aws-cn:states:::sagemaker:createTransformJob.sync', 'parameters': {'TransformJobName': 'Batch-Transform-2021-05-06-16-45', 'ModelName': 'jsfund-model-1', 'TransformInput': {'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-cn-north-1-685095924131/jsfund/input/sample_data_noheader.csv'}}, 'ContentType': 'text/csv', 'SplitType': 'Line'}, 'TransformOutput': {'S3OutputPath': 's3://sagemaker-cn-north-1-685095924131/jsfund/output/Batch-Transform-2021-05-06-08-44-23', 'AssembleWith': 'Line', 'Accept': 'text/csv'}, 'TransformResources': {'InstanceCount': 1, 'InstanceType': 'ml.m4.xlarge'}, 'DataProcessing': {'InputFilter': '$[1:72]', 'OutputFilter': '$[0,-1]', 'JoinSource': 'Input'}}, 'type': 'Task'}


In [110]:
workflow_definition = steps.Chain([
    etl_step,
    transform_step
])

# 7. 运行上述workflow，指定workflow name

In [114]:
workflow = Workflow(
    name='MyInferenceRoutine01',
    definition=workflow_definition,
    role=workflow_execution_role,
)

In [115]:
workflow.render_graph()

In [116]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws-cn:states:cn-north-1:685095924131:stateMachine:MyInferenceRoutine003'

In [117]:
execution = workflow.execute()

[INFO] Workflow execution started successfully on AWS Step Functions.


In [118]:
execution.render_progress()

In [97]:
execution.list_events(html=False)

[{'timestamp': datetime.datetime(2021, 5, 4, 15, 49, 29, 684000, tzinfo=tzlocal()),
  'type': 'ExecutionStarted',
  'id': 1,
  'previousEventId': 0,
  'executionStartedEventDetails': {'input': '{}',
   'inputDetails': {'truncated': False},
   'roleArn': 'arn:aws-cn:iam::685095924131:role/StepFunctionsWorkflowExecutionRole'}},
 {'timestamp': datetime.datetime(2021, 5, 4, 15, 49, 29, 722000, tzinfo=tzlocal()),
  'type': 'TaskStateEntered',
  'id': 2,
  'previousEventId': 0,
  'stateEnteredEventDetails': {'name': 'Extract, Transform, Load',
   'input': '{}',
   'inputDetails': {'truncated': False}}},
 {'timestamp': datetime.datetime(2021, 5, 4, 15, 49, 29, 722000, tzinfo=tzlocal()),
  'type': 'TaskScheduled',
  'id': 3,
  'previousEventId': 2,
  'taskScheduledEventDetails': {'resourceType': 'glue',
   'resource': 'startJobRun.sync',
   'region': 'cn-north-1',
   'parameters': '{"JobName":"glue-demo-job-0223"}'}},
 {'timestamp': datetime.datetime(2021, 5, 4, 15, 49, 29, 750000, tzinfo=tzlo

In [66]:
workflow.list_executions(html=True)

Name,Status,Started,End Time
16d911b3-96b8-4a8c-80ee-30cfade88a5d,SUCCEEDED,"May 06, 2021 08:46:56.986 AM","May 06, 2021 08:55:48.305 AM"


In [67]:
Workflow.list_workflows(html=True)

Name,Creation Date
HelloWorld,"Mar 06, 2020 03:32:43.495 PM"
MyInferenceRoutine01,"May 06, 2021 08:46:54.609 AM"
